In [1]:
import sys, os
import json
import argparse
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import plotly
import plotly.graph_objects as graph_obj
import plotly.figure_factory as ff

sys.path.insert(0, os.path.abspath('..'))
from utils.s3 import s3store, Connect
from utils.jsonutil import WriteDictJson, ReadDictJson
from segment.segmenttest import PrepareResults, PlotConfusion

In [2]:
output = widgets.Output()

In [3]:
def parse_arguments():
    parser = argparse.ArgumentParser(description='Process arguments')
    parser.add_argument('-credentails', type=str, default='../creds.json', help='Credentials file.')

    parser.add_argument('-model_type', type=str,  default='segmentation')
    parser.add_argument('-test_results', type=str, default='test_results.json')   

    args = parser.parse_args("")
    return args

In [4]:
with output:
    args = parse_arguments()
    print('parse_arguments()={}'.format(args))

In [5]:
with output:
    s3, creds, s3def = Connect(args.credentails)
    print('Connect()={}'.format(s3def))

In [6]:
with output:
    test_path = '{}/{}/{}'.format(s3def['sets']['test']['prefix'], args.model_type, args.test_results)
    test_data = s3.GetDict(s3def['sets']['test']['bucket'], test_path)
    print('test_data()={}'.format(test_data))

In [7]:
test_names, overview, results  = PrepareResults(test_data)
display(pd.DataFrame(overview).T)

,test date,test images,mean IoU,inference time
1: segment_deeplabv3_512x442_20211030_01,"11/12/2021, 08:45:16",5000,0.709993,0.006181
2: segment_deeplabv3_512x442_20211107_00,"11/12/2021, 08:55:08",5000,0.790635,0.006192
3: segment_deeplabv3_512x442_20211101_00,"11/12/2021, 09:05:46",5000,0.782202,0.006293
4: segment_deeplabv3_512x442_20211112_00,"11/13/2021, 06:14:40",5000,0.803974,0.006058
5: segment_deeplabv3_512x442_20211112_00,"11/14/2021, 06:50:10",5000,0.809978,0.005556
6: segment_deeplabv3_512x442_20211112_00,"11/15/2021, 07:02:34",144,0.767238,0.0065
7: segment_nas_512x442_202111141_01,"11/15/2021, 09:05:08",5000,0.73669,0.006011
8: segment_nas_512x442_20211101_00,"11/15/2021, 09:39:23",5000,0.727587,0.00609
9: segment_nas_512x442_20211029_00,"11/15/2021, 09:45:31",5000,0.700374,0.006111
10: segment_nas_512x442_20211028,"11/15/2021, 09:51:34",5000,0.624807,0.006104


In [8]:
confusion_display = graph_obj.FigureWidget()
if len(test_data) > 0:
    plot = PlotConfusion(test_data[0]['objects'], test_data[0]['results']['confusion'])
    confusion_display = graph_obj.FigureWidget(plot)

In [9]:
def SelectTest(change, output, confusion, results):
    with output:
        print('SelectTest() change={}'.format(change))
        with confusion.batch_update():
            
            c = np.array(results[change.new]['confusion'])
            norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
            confusion_text = [str(round(x, 3)) for x in norm_confusion.flatten()]
            confusion.data[0].z =  norm_confusion
            confusion.update_layout(yaxis_autorange="reversed")
            
            for i in range(len(confusion_text)):
                confusion.layout.annotations[i].text = confusion_text[i]
                
test_select = widgets.Select(
    options=test_names,
    description='Test:',
    disabled=False,
    #rows=25,
)

In [10]:
def ClearOutput(b, output):
    print('In ClarOutput')
    output.clear_output()
    
clear_output = widgets.Button(description='Clear Output')


In [11]:
display_results = widgets.HBox([test_select, confusion_display])
segment_interactive = widgets.VBox([display_results, clear_output, output])

display(segment_interactive)

clear_output.on_click(lambda b: ClearOutput(b, output=output))
test_select.observe (lambda change:SelectTest(change, 
                                            output=output, 
                                            confusion=confusion_display, 
                                            results=results), 
                    names="value")

if len(test_names) > 0:
    test_select.options = test_names
    test_select.value = test_names[0]